aula 03 coletando dados



In [21]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('database.db')

In [22]:
# consulta dos dados nos bancos de dados
consulta_atividade = """
    SELECT
        *
    FROM
       flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa.loyalty_number = flh.loyalty_number)

"""

df_atividade = pd.read_sql_query(consulta_atividade, conn)



df_atividade.head()

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


4.0 Preparação dos dados

In [23]:
# numeros de dados faltantes de cada coluna
df_atividade.isna().sum()

#selecionar somente as colunas que contem numweros
colunas = ["year",	"month", "flights_booked", "flights_with_companions",
           "total_flights", "distance", "points_accumulated","clv", "salary", "loyalty_card"]

df_colunas_numericas = df_atividade.loc[:, colunas]

#remover as linhas quw contem dados faltantes
df_dados_completos = df_colunas_numericas.dropna()
#verificar se exisatemn  daDOS faltasntes

df_dados_completos.isna().sum()

year                       0
month                      0
flights_booked             0
flights_with_companions    0
total_flights              0
distance                   0
points_accumulated         0
clv                        0
salary                     0
loyalty_card               0
dtype: int64

In [24]:
df_dados_completos.shape[0]

302952

In [25]:
df_atividade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loyalty_number               405624 non-null  int64  
 1   year                         405624 non-null  int64  
 2   month                        405624 non-null  int64  
 3   flights_booked               405624 non-null  int64  
 4   flights_with_companions      405624 non-null  int64  
 5   total_flights                405624 non-null  int64  
 6   distance                     405624 non-null  int64  
 7   points_accumulated           405624 non-null  float64
 8   points_redeemed              405624 non-null  int64  
 9   dollar_cost_points_redeemed  405624 non-null  int64  
 10  loyalty_number               405624 non-null  int64  
 11  country                      405624 non-null  object 
 12  province                     405624 non-null  object 
 13 

In [26]:
df_dados_completos.head()

,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,clv,salary,loyalty_card
0,2017,1,3,0,3,1521,152.0,7919.20,92552.0,Aurora
3,2017,1,0,0,0,0,0.0,4170.57,63253.0,Star
4,2017,1,0,0,0,0,0.0,6622.05,91163.0,Star
5,2017,1,0,0,0,0,0.0,48356.96,70323.0,Nova
6,2017,1,0,0,0,0,0.0,5143.88,76849.0,Nova


5.0 machine learning

In [27]:
from sklearn import tree as tr

x_atributos =df_dados_completos.drop(columns="loyalty_card")
y_rotulos = df_dados_completos.loc[:, "loyalty_card"]

#definicao do algortimo
modelo = tr.DecisionTreeClassifier(max_depth=5)

#treinamento do algortimo
modelo_treinado = modelo.fit(x_atributos, y_rotulos)

In [28]:
tr.plot_tree(modelo_treinado, filled=True);

6.0 apresentando um resultado

In [29]:
y_rotulos.head()

0    Aurora
3      Star
4      Star
5      Nova
6      Nova
Name: loyalty_card, dtype: object

In [30]:
x_novo = x_atributos.sample()
previsao = modelo_treinado.predict_proba(x_novo)

print("Prob - Aurora: {:.2f}% - nova: {:.2f}% - Star: {:.2f}%".format(100*previsao[0][0], 100*previsao[0][1], 100*previsao[0][2]))

Prob - Aurora: 0.00% - nova: 67.43% - Star: 32.57%


In [31]:
x_atributos.head()

,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,clv,salary
0,2017,1,3,0,3,1521,152.0,7919.20,92552.0
3,2017,1,0,0,0,0,0.0,4170.57,63253.0
4,2017,1,0,0,0,0,0.0,6622.05,91163.0
5,2017,1,0,0,0,0,0.0,48356.96,70323.0
6,2017,1,0,0,0,0,0.0,5143.88,76849.0


In [32]:
x_atributos.loc[:,'year'].min()

2017

In [34]:
x_atributos.loc[:,'year'].max()

2018

7.0 Painel de visualização

In [ ]:
!pip install gradio

In [35]:
import gradio as gr
import numpy as np

In [ ]:
def predict(*args):
  x_novo = np.array([args]).reshape(1, -1)
  previsao = modelo_treinado.predict_proba(x_novo)

  return{"Aurora":previsao[0][0], "Nova":previsao[0][1], "Star":previsao[0][2] }




with gr.Blocks() as demo:
  # titulo do painel
  gr.Markdown(""" # Propensao de compra""")

  with gr.Row():
    with gr.Column():
      gr.Markdown(""" # Atributos do cliente""")
      year                        = gr.Slider( label ="year", minimum=2017, maximum=2018, step=1, randomize=True)
      month                       = gr.Slider( label ="month", minimum=1, maximum=12, step=1, randomize=True)
      flights_booked              = gr.Slider( label ="flights_booked", minimum=0, maximum=21, step=1, randomize=True)
      flights_with_companions     = gr.Slider( label ="flights_with_companions", minimum=0, maximum=11, step=1, randomize=True)
      total_flights               = gr.Slider( label ="total_flights", minimum=0, maximum=32, step=1, randomize=True)
      distance                    = gr.Slider( label ="distance", minimum=0, maximum=6293, step=1, randomize=True)
      points_accumulated          = gr.Slider( label ="points_accumulated", minimum=0.0, maximum=676.5, step=1, randomize=True)
      salary                      = gr.Slider( label ="salary", minimum=-58486.0, maximum=407228.0, step=1, randomize=True)
      clv                         = gr.Slider( label ="clv", minimum=2119.89, maximum=83325.38, step=1, randomize=True)

      with gr.Row():
        gr.Markdown(""" # Linha 1""")

        with gr.Row():
          gr.Markdown("""# Botao de previsao""")
          predict_btn = gr.Button(value = "Previsao")

    with gr.Column():
      gr.Markdown(""" # Propensao de compra do cliente""")
      label = gr.Label()

    # botao predict
    predict_btn.click(
        fn=predict,
        inputs =[
            year,
            month,
            flights_booked,
            flights_with_companions,
            total_flights,
            distance,
            points_accumulated,
            clv,
            salary
            ],
        outputs=[label])

demo.launch(debug=True)


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a023a05bee497f5f9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
